In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-13b-chat-hf")

In [7]:
df

,tweet_id,human_written_reply
0,1375154120106074114,"This is not correct. Actually, mRNA vaccine wi..."
1,1373234271201206274,It is a conspiracy theory. Bill Gates has noth...
2,1351618808616787968,You description about the vaccine is not accur...
3,1407787388924006401,The evidence you mentioned is wrong. The vacci...
4,1351430882867994625,Please stop this misinformation. What you are ...
...,...,...
586,1374556326504894465,Your highlighted claim is not correct. As debu...
587,1375085164192206855,Sorry to see your misleading post. I understan...
588,1395036936218284035,"Well, your mentioned microchip is not correct...."
589,1395959987726274561,Your claims are inaccurate. The covid19 is not...


In [8]:
DATA2 = r"C:\Users\nashe\Box\Research\misinfo-prj\datasets\social_media_data.pkl"

In [9]:
df2 = pd.read_pickle(DATA2)
df2

,reply_id,tweet_id,refutation,evidence,politeness
0,1335913130711371777,1335892654832320513,1,0,0
1,1336107858140803074,1335916733148913665,1,0,0
2,1335961370127380488,1335916733148913665,0,0,1
3,1335988879921442818,1335977737719308294,0,0,0
4,1335982116526546945,1335977737719308294,1,0,0
...,...,...,...,...,...
749,1411545172903534595,1411545172903534595,0,1,0
750,1412069858464894980,1412068692876275718,1,1,1
751,1412581535836872706,1412559319577501697,0,1,0
752,1412566724453470208,1412559319577501697,1,0,0


In [10]:
import os

os.environ['token_secret']

keys_dict = {
    "consumer_key": "1vZkVcRCOPGMOpUAwgOo0bK5M",
    "consumer_secret": "biGatZTZFRLUmCj2eIWxkPlSw9NuyoqysuLk9iIkDsM8pV4Ov4",
    "access_token": "3060923399-lAW8324utfSjXMBvDBGNIvyjbusFWCzQXnRE4Vq",
    "access_token_secret": "yaMq7IU3BGJsO9MhIr9by5elF9qkiqidkz3VwdWLTaI27",
    "bearer_token": "AAAAAAAAAAAAAAAAAAAAAFJL%2FwAAAAAAH3WGbVUQedhKAR0sKobDPxdeIJI%3Db7jtjlvSc7MAwzkfV4Eo5Yc2ycxiaCv9xfOPMvWWtHyCf7HYpN"
}

In [11]:
from tweepy import Client

twitter_client = Client(
    **keys_dict
    # bearer_token=os.environ['bearer_token'],
    # consumer_key=os.environ['consumer_key'],
    # consumer_secret=os.environ['consumer_secret'],
    # access_token=os.environ['access_token'],
    # access_token_secret=os.environ['token_secret']
)

In [ ]:
twitter_client.get_tweets('1375154120106074114', user_auth=False)

In [9]:
os.environ['bearer_token']

'AAAAAAAAAAAAAAAAAAAAAFPnowEAAAAAbVigRpkg6JtWaf9cKqdhnE4iF80varsDGDZVtHNn2sBPTkH6SC2kvrysPm0bb6KA2rNkDUnuVUBzIHFMlf'

In [13]:
import tweepy
from tweepy import NotFound, Forbidden

from tqdm import tqdm

keys_dict = {
    "api_key": "1vZkVcRCOPGMOpUAwgOo0bK5M",
    "api_key_secret": "biGatZTZFRLUmCj2eIWxkPlSw9NuyoqysuLk9iIkDsM8pV4Ov4",
    "access_token": "3060923399-lAW8324utfSjXMBvDBGNIvyjbusFWCzQXnRE4Vq",
    "access_token_secret": "yaMq7IU3BGJsO9MhIr9by5elF9qkiqidkz3VwdWLTaI27",
    "bearer_token": "AAAAAAAAAAAAAAAAAAAAAFJL%2FwAAAAAAH3WGbVUQedhKAR0sKobDPxdeIJI%3Db7jtjlvSc7MAwzkfV4Eo5Yc2ycxiaCv9xfOPMvWWtHyCf7HYpN"
}

# instantiating the api
auth = tweepy.OAuthHandler(keys_dict["api_key"], keys_dict["api_key_secret"])
auth.set_access_token(keys_dict["access_token"], keys_dict["access_token_secret"])

# creating API object
api = tweepy.API(auth, wait_on_rate_limit=True)

ids = []
tweets = []
replies = []
for idx, row in tqdm(df2.iterrows(),  total=df2.shape[0]):
    try:
        tweet = api.get_status(row['tweet_id'], tweet_mode = "extended")._json['full_text']
        reply = api.get_status(row['reply_id'], tweet_mode = "extended")._json['full_text']
    except (NotFound, Forbidden): continue

    ids.append((row['tweet_id'], row['reply_id']))
    tweets.append(tweet)
    replies.append(reply)


  0%|          | 0/754 [00:00<?, ?it/s]

100%|██████████| 754/754 [26:13<00:00,  2.09s/it]   


In [14]:
# Make a tsv

social_df = pd.DataFrame(columns=['tweet_id', 'tweet', 'reply_id', 'reply'])

social_df['tweet_id'] = [i[0] for i in ids]
social_df['tweet'] = tweets
social_df['reply_id'] = [i[1] for i in ids]
social_df['reply'] = replies

social_df.to_csv(
    r"C:\Users\nashe\Box\Research\misinfo-prj\datasets\scraped_socialmedia_data.tsv",
    sep='\t',
    index=False
)

In [15]:
crowd_ids = []
crowd_tweets = []
for idx, row in tqdm(df.iterrows(),  total=df.shape[0]):
    try:
        tweet = api.get_status(row['tweet_id'], tweet_mode = "extended")._json['full_text']
    except (NotFound, Forbidden): continue

    crowd_ids.append(row['tweet_id'])
    crowd_tweets.append(tweet)


100%|██████████| 591/591 [02:01<00:00,  4.88it/s]


In [20]:
# Make a tsv

crowd_df = pd.DataFrame(columns=['tweet_id', 'tweet'])

crowd_df['tweet_id'] = crowd_ids
crowd_df['tweet'] = crowd_tweets
crowd_df.to_csv(
    r"C:\Users\nashe\Box\Research\misinfo-prj\datasets\scraped_crowdsource_data.tsv",
    sep='\t',
    index=False
)

In [23]:
twt

{'created_at': 'Sat May 22 04:29:31 +0000 2021',
 'id': 1395959987726274561,
 'id_str': '1395959987726274561',
 'full_text': 'Covid19 was a bioweapon &amp; so is the vaccine! First they killed people w Covid. Then they scared them into taking a vaccine. Now the vaccine is shedding into the general public. Ultimately the population will be greatly reduced thru Covid deaths, vaccine deaths &amp; sterilization.',
 'truncated': False,
 'display_text_range': [0, 285],
 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 16456394,
  'id_str': '16456394',
  'name': 'Dr. Shannon Kroner',
  'screen_name': 'drshannonkroner',
  'location': 'Los Angeles, CA',
  'description': 'Executive Director of